In [19]:
#install packages not on instance
#!pip install xgboost

In [20]:
#Cleaning Data
import os
import pandas as pd
import numpy as np
import datetime
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
#modeling tools
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
#modeling
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
#import xgboost as xgb
#sagemaker training
import sagemaker
#deploying model
from sklearn.externals import joblib
import pickle

In [21]:
#import data
outage = pd.read_csv('We_WPS_comments_categories.csv')
outage.head()

,outage category,Mobile Data Remarks,outage category number
0,Equipment,ON LtOut TRBL SAYS OTHER SEE COMMENTS,0
1,Equipment,ON LtOut TRBL SAYS ONE LEG OF URD IS DEAD CUST...,0
2,Vegetation,ON HAZ TRBL SAYS REMOVED TREE FROM LINE NEARPO...,7
3,Equipment,ON LtOut TRBL SAYS REPAIRED BAD PRIMARY URD CA...,0
4,Equipment,ON LtOut TRBL SAYS BAD CONNECTOR ON CUST SERV ...,0


In [22]:
#seperating out unknown causes
all_unknowns = outage[outage['outage category'] == 'Unknown']
#save off unknowns
all_unknowns.to_csv('Unknown Outcomes.csv')
#remove unknowns and not reporteable
outage = outage[outage['outage category'] != 'Unknown']
outage = outage[outage['outage category'] != 'Not Reportable']

# Training

In [23]:
#split features and target
X = outage['Mobile Data Remarks']
Y = pd.factorize(outage['outage category'])[0]

In [24]:
#Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, random_state=53)
#Initialize a CountVectorizer object
count_vectorizer = CountVectorizer(stop_words='english')
#Transform the training data using only the 'text' column values
X_train = count_vectorizer.fit_transform(X_train)
#Transform the test data using only the 'text' column values
X_test = count_vectorizer.transform(X_test)
#export count vectorizer
pickle.dump(count_vectorizer, open("count_vectorizer.pickle", "wb"))

In [26]:
outage.iloc[:, 1] = outage.iloc[:, 1].map(lambda x: x.lower())
outage.head()

,outage category,Mobile Data Remarks,outage category number
0,Equipment,on ltout trbl says other see comments,0
1,Equipment,on ltout trbl says one leg of urd is dead cust...,0
2,Vegetation,on haz trbl says removed tree from line nearpo...,7
3,Equipment,on ltout trbl says repaired bad primary urd ca...,0
4,Equipment,on ltout trbl says bad connector on cust serv ...,0


In [28]:
outage.iloc[:, 0] = outage.iloc[:, 0].values.astype('unicode')
#make sure all text is lowercase
X_train = outage.iloc[:, 0].map(lambda x: x.lower())
X_train.head()

0     equipment
1     equipment
2    vegetation
3     equipment
4     equipment
Name: outage category, dtype: object

In [ ]:
#set up models
rf = RandomForestClassifier(n_estimators = 500, max_features = None, n_jobs=-1)
rf = rf.fit(X_train, y_train)

In [ ]:
#train
rf = rf.fit(X_train, y_train)

# Test

In [ ]:
#model performance
y_pred = rf.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
#which outcomes are being predicted best
confusion_matrix(y_test, y_pred)

In [ ]:
X = count_vectorizer.transform(X)
X[0]

In [ ]:
# see how it categoizes unknonwn outcome
prediction = model.predict(X[0])
#import outcome decoder
decoder = pd.read_csv('decoder.csv')
#set index to the outcome number
decoder = decoder.set_index('Number')
#tranlate it to a dict
decoder = decoder.to_dict()
#convert prediction to text
result = decoder['Word'][prediction]
#see result
print(result)

# Deploy

In [ ]:
# Save the model as a pickle in a file 
#joblib.dump(rf, 'Text_RF.pkl')